In [1]:
import os
#os.chdir('..')
from matplotlib import pyplot as plt
import numpy as np
import tensorflow as tf
import pandas as pd
from collections import namedtuple

import virl

C:\Users\ASUS\AppData\Roaming\Python\Python36\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\ASUS\AppData\Roaming\Python\Python36\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\ASUS\AppData\Roaming\Python\Python36\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


#### Help function to see the converge

In [2]:
EpisodeStats = namedtuple("Stats",["episode_lengths", "episode_rewards"])

# This is used to compare the episode length over time,which is always 52
def plot_episode_stats(stats, smoothing_window,environment,noisy,noshow=False):

    # Plot the episode reward over time,so we can see when did it converge
    fig1 = plt.figure(figsize=(10,5))
    rewards_smoothed = pd.Series(stats.episode_rewards).rolling(smoothing_window, min_periods=smoothing_window).mean()
    plt.plot(rewards_smoothed)
    plt.xlabel("Episode")
    plt.ylabel("Episode Reward (Smoothed)")
    plt.title("Episode Reward over Time (Smoothed over window size {})".format(smoothing_window))
    plt.savefig('graphs/random_agent/converge graph/average performance vs number of episodes'+ "for random agent" + " " + environment + " " + noisy + " "+ '.png')
    if noshow:
        plt.close(fig1)
    else:
        plt.show(fig1)

    #return fig1, fig2, fig3
    return fig1

#### Train on the environment and doing fine tuning and predicting on the action to construct visualization

####  First : make the function to show the action content

In [3]:
def show_action(actions):
    outcome = " " 
    for i in range(len(actions)):
        outcome += ("week " + str("{:2}".format(i+1)) + " : ")
        if actions[i] == 0:
            outcome += ("%-17s" %("None"))
        if actions[i] == 1:
            outcome += ("%-17s" %("Full Lockdown"))
        if actions[i] == 2:
            outcome += ("%-17s" %("Track & Trace"))
        if actions[i] == 3:
            outcome += ("%-17s" %("Social Distancing"))
        if len(actions) - 1 != i:
            outcome += (" -> ")
        if (i+1) % 4 == 0:
            outcome += "\n"
    return outcome

#### Second : make the function to run one episode with the trained deterministic agent

In [4]:
def run_one_episode(env,environment,noisy):
    states = []
    rewards = []
    actions = []
    done = False
    s = env.reset()
    states.append(s)
    while not done:
        # make a step
        action=np.random.choice(env.action_space.n)
        next_state, reward, done, _ = env.step(action) # random agent

        states.append(next_state)
        rewards.append(reward)
        actions.append(action)
    fig, axes = plt.subplots(1, 2, figsize=(20, 8))
    # This is the place for visualizing the policy
    print("\n")
    print(show_action(actions))
    labels = ['s[0]: susceptibles', 's[1]: infectious', 's[2]: quarantined', 's[3]: recovereds']
    states = np.array(states)
    for i in range(4):
        axes[0].set_title("The states analysis for" + " " + environment + " " + noisy + " " + "for random agent")
        axes[0].plot(states[:,i], label=labels[i]);
        axes[0].set_xlabel('weeks since start of epidemic')
        axes[0].set_ylabel('State s(t)')
        axes[0].legend()
        axes[1].plot(rewards);
        axes[1].set_title('The Reward to each action' + " " + environment + " " + noisy + " " + "for random agent")
        axes[1].set_xlabel('weeks since start of epidemic')
        axes[1].set_ylabel('reward r(t)')
    
    print('total reward', np.sum(rewards))
    plt.savefig('graphs/random_agent/state_and_reward/state and reward for'+ " " + environment + " " + noisy + " " + "for ps_lfa" + '.png')
    return actions

#### Third : make the function to train the agent and evaluate on the trained agent by making one episode

In [5]:
def run_random(env,num_episodes):
    
    stats = EpisodeStats(
        episode_lengths=np.zeros(num_episodes),
        episode_rewards=np.zeros(num_episodes)) 
    
    for i_episode in range(num_episodes):
        done = False
        s = env.reset()
        t = 0
        while not done:
            t += 1
            s, r, done, info = env.step(action=np.random.choice(env.action_space.n)) # random agent
            stats.episode_rewards[i_episode] += r
            stats.episode_lengths[i_episode] = t
            print("\rStep {} @ Episode {}/{} ({})".format(
                    t, i_episode + 1, num_episodes, stats.episode_rewards[i_episode - 1]), end="")            
            
    return stats

In [6]:
def random_train_and_predict(env,num_episodes,environment,noisy):
    # Instantiate a FunctionApproximator (i.e. the linear function approximator)

    tf.reset_default_graph()

    global_step = tf.Variable(0, name="global_step", trainable=False)
    
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        # This is the training stage
        # policy may vary. ~600-1000 seemed to work well.
        stats = run_random(env,num_episodes)
        ## then,we use the optimized function approximator to predict on optimal policy and gain them
        actions = run_one_episode(env,environment,noisy)
        
        print("\n")
    return stats

#### Fourth : make the function to train the agent in all environment and problems and evaluate on the trained agent by making one episode¶

In [7]:
def overall_training_and_evaluation(num_episodes):
    
    '''Train for each of the environments and problems'''
    
    stats_for_problems_nnoise = []
    for i in range (10):
        env = virl.Epidemic(problem_id=i,noisy=False)
        print("This is problem" + " " + str(i) + " " + "without noise for random agent")
        # train on the problem and evaluating by making one episode
        stats0 = random_train_and_predict(env,num_episodes,"problem" + " " + str(i),"without noise")
        stats_for_problems_nnoise.append(stats0)
        
    stats_for_problems_noise = []
    for j in range (10):
        env = virl.Epidemic(problem_id=j,noisy=True)
        # train on the problem and evaluating by making one episode
        print("This is problem" + " " + str(j) + " " + "with noise for random agent")
        stats1 = random_train_and_predict(env,num_episodes,"problem" + " " + str(j),"with noise")
        stats_for_problems_noise.append(stats1)
        

    env = virl.Epidemic(stochastic=True,noisy=False)
    # train on the stochastic environment and evaluating by making one episode
    print("This is stochastic problem " +  " " + "without noise for random agent")
    stats2 = random_train_and_predict(env,num_episodes,"stochastic","without noise")
    
    env = virl.Epidemic(stochastic=True,noisy=True)
    # train on the stochastic environment and evaluating by making one episode
    print("This is stochastic problem " +  " " + "with noise for random agent")
    stats3 = random_train_and_predict(env,num_episodes,"stochastic","with noise")
    
    print("----------------------------------Learning Curves----------------------------------------------")
    return stats_for_problems_nnoise,stats_for_problems_noise,stats2,stats3

#### Fifth :Draw the graph to see how the agent learn (Learning curve)

In [8]:
def show_graphs_policy_random(stats_for_problems_nnoise,stats_for_problems_noise,stats2,stats3):
    '''Used for ploting the converging curve for each training problem and environments'''
    
    for i in range(len(stats_for_problems_nnoise)):
        plot_episode_stats(stats_for_problems_nnoise[i],100,"problem" + " " + str(i),"without noise")
        print("This is the convergence graphs for problem" + " " + str(i) + " " + "without noise for random agent")
    for j in range(len(stats_for_problems_noise)):
        plot_episode_stats(stats_for_problems_noise[j],100,"problem" + " " + str(j),"with noise")
        print("This is the convergence graphs for problem" + " " + str(j) + " " + "with noise for random agent")
        
    plot_episode_stats(stats2,100,"stochastic","without noise")
    print("This is the convergence graphs for stochastic environment" +  " " + "without noise for random agent")
    
    plot_episode_stats(stats3,100,"stochastic","with noise")
    print("This is the convergence graphs for stochastic environment" +  " " + "with noise for random agent")


####  The overall visualization for all of the environment and problems for passing to run_eval.ipynb file

In [9]:
def visualization_for_random_agent():
    # Train and evaluate as well as drawing state graph and reward graph
    stats_for_problems_nnoise,stats_for_problems_noise,stats2,stats3 = overall_training_and_evaluation(2000)
    # Plot the converging curve
    show_graphs_policy_random(stats_for_problems_nnoise,stats_for_problems_noise,stats2,stats3)

In [10]:
#visualization_for_random_agent()